In [15]:
import sys
sys.path.insert(0, '/Users/shreemit/Developer/movie-recs')
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import pandas as pd
from filters.colab_filter import CollaborativeMovieRecommender
from filters.content_based import ContentBasedRecommender


/Users/shreemit/miniforge3/envs/leaseGPT/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
test_data = pd.read_csv('/Users/shreemit/Developer/movie-recs/test_data.csv')

In [28]:
def recommend_movies(user_id, ratings_path, movies_path, movie_features_path, method='colab_filter'):
    ratings = pd.read_csv(ratings_path)
    movies = pd.read_csv(movies_path)
    movie_features = pd.read_pickle(movie_features_path)

    # get top-rated movies watched by user
    user_ratings = ratings[ratings['userId'] == user_id]
    user_ratings = user_ratings.sort_values('rating', ascending=False)
    user_top3 = user_ratings.head(3)

    movies_to_rec = set()

    if method == 'content_based':
        content_based_recommender = ContentBasedRecommender(movies, movie_features)
        for movieId in user_top3['movieId'].values:
            movies_to_rec.update(content_based_recommender.get_content_based_recommendations(movie_id=movieId,
                                                                                             n_recommendations=5 ))
    elif method == 'colab_filter':
        colab_filter = CollaborativeMovieRecommender(movies, ratings)
        colab_filter.preprocess_data()
        colab_filter.create_X()

        # find similar movies for the top-rated movies watched by the user
        colab_filter_movies = set()
        for movieId in user_top3['movieId'].values:
            movies_to_rec.update(colab_filter.find_similar_movies(movie_id=movieId, k=5))
        
    # remove the movies that the user has already seen
    movies_to_rec = movies_to_rec - set(user_ratings['movieId'].values)
    
    return list(movies_to_rec)

In [29]:
recommended_movies = recommend_movies(user_id=2, 
                                      ratings_path='/Users/shreemit/Developer/movie-recs/train_data.csv', 
                                      movies_path='/Users/shreemit/Developer/movie-recs/merge_movies.csv', 
                                      movie_features_path='/Users/shreemit/Developer/movie-recs/movie_features.pkl')
print(recommended_movies)

[65577, 1961, 1257, 2959, 7153, 62553, 71579, 2716, 86815]


In [12]:
recommended_movies = recommend_movies(user_id=2, 
                                      ratings_path='/Users/shreemit/Developer/movie-recs/train_data.csv', 
                                      movies_path='/Users/shreemit/Developer/movie-recs/merge_movies.csv', 
                                      movie_features_path='/Users/shreemit/Developer/movie-recs/movie_features.pkl')
print(recommended_movies)

[65577, 1961, 1257, 2959, 7153, 62553, 71579, 2716, 86815]


In [64]:
# looping through the test data to get the recommended movies for each user
results_colab = []
for user_id in tqdm(test_data['userId'].unique()):
    recommended_movies = recommend_movies(user_id=user_id, 
                                          ratings_path='/Users/shreemit/Developer/movie-recs/train_data.csv', 
                                          movies_path='/Users/shreemit/Developer/movie-recs/merge_movies.csv', 
                                          movie_features_path='/Users/shreemit/Developer/movie-recs/movie_features.pkl')
    results_colab.append({'userId': user_id, 'movieId': recommended_movies})

results_df = pd.DataFrame(results_colab)

  0%|          | 0/610 [00:00<?, ?it/s]

In [30]:
results_content = []
for user_id in tqdm(test_data['userId'].unique()):
    recommended_movies = recommend_movies(method='content_based',
                                          user_id=user_id, 
                                          ratings_path='/Users/shreemit/Developer/movie-recs/train_data.csv', 
                                          movies_path='/Users/shreemit/Developer/movie-recs/merge_movies.csv', 
                                          movie_features_path='/Users/shreemit/Developer/movie-recs/movie_features.pkl')
    results_content.append({'userId': user_id, 'movieId': recommended_movies})

  0%|          | 0/610 [00:00<?, ?it/s]

In [24]:
for id in results:
    print(id)
    print(id['userId'])
    break

{'userId': 1, 'movieId': [2, 932, 5796, 165, 5989, 115210, 2762, 1430, 344]}
1


In [51]:
# preparing dict for test data {userId: [movieId]}
test_dict = {}
for user_id in test_data['userId'].unique():
    test_dict[user_id] = test_data[test_data['userId'] == user_id]['movieId'].values


AxisError: axis 1 is out of bounds for array of dimension 1

In [145]:
len(test_dict)

610

In [150]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [200]:
def evaluate_recommendations(results_content, test_data):

    if len(results_content) != len(test_data):
        print("Error: Lengths of 'recommended_movies' and 'test_data' do not match.")
    binary_recommendations = np.array([])
    for result in results_content:
        test_data = set(test_dict[result['userId']])
        # binary_recommendation = [int(movie in test_data) for movie in result['movieId']]
        for movie in result['movieId']:
                # print(result['userId'])
            binary_recommendation = [int(movie in test_data)]
            if movie in test_data:
                break
        binary_recommendations = np.append(binary_recommendations,binary_recommendation)
        
    print(np.count_nonzero(binary_recommendations))
    # Calculate precision, recall, and accuracy
    true_positives = np.sum(binary_recommendations)
    predicted_positives = len(binary_recommendations)
    actual_positives = len(test_data)

    precision = true_positives / predicted_positives
    recall = true_positives / actual_positives
    accuracy = true_positives / len(results_content)

    # Display the results
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Accuracy: {accuracy}')


    return binary_recommendations

In [201]:
res = evaluate_recommendations(results_content, test_dict)

129
Precision: 0.21147540983606558
Recall: 0.19815668202764977
Accuracy: 0.21147540983606558


In [202]:
res = evaluate_recommendations(results_colab, test_dict)

395
Precision: 0.6475409836065574
Recall: 0.6067588325652842
Accuracy: 0.6475409836065574


In [195]:
np.count_nonzero(res)

395

In [37]:
(test_data['userId'].nunique()), len(results_content)

(610, 610)

In [99]:
results_colab

[{'userId': 1, 'movieId': [2, 932, 5796, 165, 5989, 115210, 2762, 1430, 344]},
 {'userId': 2,
  'movieId': [65577, 1961, 1257, 2959, 7153, 62553, 71579, 2716, 86815]},
 {'userId': 3, 'movieId': [163937, 97188, 99437, 99030, 98623]},
 {'userId': 4, 'movieId': [2183, 3246, 2066, 2034, 3545, 2970, 2204, 2143]},
 {'userId': 5,
  'movieId': [1060,
   2762,
   2571,
   1196,
   364,
   6062,
   150,
   374,
   6041,
   4350,
   6334,
   158,
   6335]},
 {'userId': 6,
  'movieId': [144352, 153, 96520, 10, 587, 588, 364, 2927, 434, 377, 442]},
 {'userId': 7,
  'movieId': [74624,
   480,
   60514,
   3107,
   72294,
   296,
   3243,
   2571,
   2126,
   47,
   3698,
   86644,
   3513]},
 {'userId': 8,
  'movieId': [421, 457, 588, 590, 1616, 17, 1042, 434, 21, 185, 508]},
 {'userId': 9,
  'movieId': [5954,
   47044,
   260,
   5989,
   5577,
   1291,
   2571,
   6157,
   6548,
   46965,
   6006,
   5016]},
 {'userId': 10, 'movieId': [60684, 102445, 71535, 94864, 103249]},
 {'userId': 11,
  'movi

In [123]:
for result in results_colab:
    test_set = set(test_dict[result['userId']])
    for movie in result['movieId']:
        if movie in test_set:
            print(result['userId'])
        

4
5
5
6
6
6
6
6
6
6
7
8
8
8
8
13
15
15
17
18
18
20
20
21
21
22
22
23
23
23
24
25
25
25
25
25
26
27
27
28
30
30
30
30
30
31
32
34
34
37
37
37
39
39
40
40
40
41
41
41
41
43
44
44
44
45
45
45
46
48
49
50
52
52
54
54
54
54
54
56
56
56
57
57
58
58
59
61
62
62
63
63
64
64
64
64
65
66
66
68
68
69
71
71
72
73
74
75
75
76
76
76
77
77
77
77
77
77
80
80
81
81
81
82
84
84
86
86
86
86
88
88
88
91
91
91
93
93
94
94
95
95
95
97
99
99
102
102
102
104
105
105
106
107
109
109
109
111
112
112
112
112
113
114
114
114
115
115
116
116
117
117
117
117
118
119
119
121
121
121
122
122
123
123
123
124
125
125
126
126
128
129
130
130
131
131
132
132
132
132
133
133
134
134
134
134
135
135
135
135
135
136
136
138
139
139
139
139
141
142
142
144
144
145
149
150
151
152
153
153
153
159
159
160
160
160
160
160
161
161
164
164
165
166
166
166
167
170
170
170
171
171
171
171
174
174
176
176
177
177
178
178
179
181
181
183
184
185
187
188
188
189
189
190
190
191
191
192
193
195
195
196
196
198
198
198
199
199
200
201
2

In [93]:
# manually checking the results
for result in results_content[0]['movieId']:
    movie = movies[movies['movieId'] == result]
    print(result)
    print(str(movie['title']))

737
595    Barb Wire
Name: title, dtype: object
738
Series([], Name: title, dtype: object)
33
Series([], Name: title, dtype: object)
1412
1087    Some Mother's Son
Name: title, dtype: object
326
284    To Live
Name: title, dtype: object
102
90    Mr. Wrong
Name: title, dtype: object
5901
4116    Empire
Name: title, dtype: object
3438
2570    Teenage Mutant Ninja Turtles
Name: title, dtype: object
211
179    Browning Version, The
Name: title, dtype: object
1684
1269    Mrs. Dalloway
Name: title, dtype: object
1654
1244    FairyTale: A True Story
Name: title, dtype: object
86
77    White Squall
Name: title, dtype: object
3961
Series([], Name: title, dtype: object)
29
28    City of Lost Children, The
Name: title, dtype: object
734
Series([], Name: title, dtype: object)


In [87]:
results_content[0]['movieId']

[737, 738, 33, 1412, 326, 102, 5901, 3438, 211, 1684, 1654, 86, 3961, 29, 734]